In [1]:
import pandas as pd
import json
import re

In [2]:
# single_train_data = pd.read_csv('../data/trainData/ILDC_single_train_dev.csv')
data = pd.read_csv('trainData/ILDC_multi_train_dev.csv').reset_index()
data = data.rename(columns={
    'index': 'article_number',
    'text' :'text',
    'label': 'label',
    'split': 'split'
})
data

,article_number,text,label,split
0,0,"F. NARIMAN, J. Leave granted. In 2008, the Pu...",1,train
1,1,"S. THAKUR, J. Leave granted. These appeals ar...",0,train
2,2,"Markandey Katju, J. Leave granted. Heard lear...",1,train
3,3,"ALTAMAS KABIR,J. Leave granted. The question ...",1,train
4,4,"CIVIL APPEAL NO. 598 OF 2007 K. MATHUR, J. Th...",1,train
...,...,...,...,...
6071,6071,civil appellate jurisdiction civil appeal numb...,1,dev
6072,6072,criminal appellate jurisdiction special leave\...,0,dev
6073,6073,civil appellate jurisdiction civil appeal numb...,0,dev
6074,6074,civil appellate jurisdiction civil appeal numb...,1,dev


#### Articles for Indian constitution, Penal Code and Code of Criminal Procedure

In [3]:
constitution = pd.read_csv('constitution_of_india.csv')
constitution

,Articles
0,"1. Name and territory of the Union\n(1) India,..."
1,1. The territories of the States; the Union te...
2,2. Admission or establishment of new States: P...
3,2A. Sikkim to be associated with the Union Rep...
4,3. Formation of new States and alteration of a...
...,...
451,378A. Special provision as to duration of Andh...
452,392. Power of the President to remove difficul...
453,393. Short title This Constitution may be call...
454,"394. Commencement This article and Articles 5,..."


In [4]:
number = []
for article in constitution['Articles']:
    number.append(article.split('.')[0])
constitution['number'] = number
constitution

,Articles,number
0,"1. Name and territory of the Union\n(1) India,...",1
1,1. The territories of the States; the Union te...,1
2,2. Admission or establishment of new States: P...,2
3,2A. Sikkim to be associated with the Union Rep...,2A
4,3. Formation of new States and alteration of a...,3
...,...,...
451,378A. Special provision as to duration of Andh...,378A
452,392. Power of the President to remove difficul...,392
453,393. Short title This Constitution may be call...,393
454,"394. Commencement This article and Articles 5,...",394


In [5]:
import re
def concat_strings(group):
    template = group.iloc[0]
    article_list = []
    for i, row in group.iterrows():
        actual_text = row['Articles'].index('.')
        article_text = re.sub(r'\(\d*\)', '', row['Articles'][actual_text+2:]).replace('\n', '.')
        article_list.append(article_text)
    template['final_Articles'] = '. '.join(article_list)
    return template

constitution = constitution.groupby(by=['number']).apply(concat_strings)
constitution

,Articles,number,final_Articles
number,,,
1,"1. Name and territory of the Union\n(1) India,...",1,"Name and territory of the Union. India, that i..."
10,10. Continuance of the rights of citizenship E...,10,Continuance of the rights of citizenship Every...
100,"100. Voting in Houses, power of Houses to act ...",100,"Voting in Houses, power of Houses to act notwi..."
101,101. Vacation of seats\n(1) No person shall be...,101,Vacation of seats. No person shall be a member...
102,102. Disqualification s for membership\n(1) A ...,102,Disqualification s for membership. A person sh...
...,...,...,...
95,95. Power of the Deputy Speaker or other perso...,95,Power of the Deputy Speaker or other person to...
96,96. The Speaker or the Deputy Speaker not to p...,96,The Speaker or the Deputy Speaker not to presi...
97,97. Salaries and allowances of the Chairman an...,97,Salaries and allowances of the Chairman and De...


In [6]:
crpc = pd.read_csv('full_crpc.csv')
crpc['article'] = crpc['article'].apply(lambda x: x.replace('.', ''))
crpc

,article,full_text
0,1,Short title extent and commencement. This Act ...
1,2,"Definitions. In this Code, unless the context ..."
2,3,"Construction of references. In this Code,- In ..."
3,4,Trial of offences under the Indian Penal Code ...
4,5,"Saving. Nothing contained in this Code shall, ..."
...,...,...
497,480,Practising pleader not to sit as Magistrate in...
498,481,Public servant concerned in sale not to purcha...
499,482,Saving of inherent powers of High Court. Nothi...
500,483,Duty of High Court to exercise continuous supe...


In [7]:
IPC_file = open('IPC.json')

IPC = pd.DataFrame(json.load(IPC_file))
IPC

,section_number,section_text
0,1,This Act shall be called the Indian Penal Code...
1,2,Every person shall be liable to punishment und...
2,3,"Any person liable, by any Indian law, to be tr..."
3,4,The provisions of this Code apply also to any ...
4,5,Nothing in this Act shall affect the provision...
...,...,...
554,507,Whoever commits the offence of criminal intimi...
555,508,Whoever voluntarily causes or attempts to caus...
556,509,"Whoever, intending to insult the modesty of an..."
557,510,"Whoever, in a state of intoxication, appears i..."


## Modify Data

In [8]:
IPC_number = []
CrPC_number = []
Constitution_number = []
for text in data['text']:
    lower_text=text.replace('.', '').lower()
    if 'crpc' in lower_text or 'code of criminal' in lower_text or 'criminal procedure' in lower_text:
        CrPC_number.append(set(re.findall(r'(?<=crpc\s)[0-9]+|[0-9]+(?=\scrpc) | '
                                           r'(?<=of the code of criminal\s)[0-9]+|[0-9]+(?=\sof the code of criminal) | '
                                           r'(?<=of the criminal procedure\s)[0-9]+|[0-9]+(?=\sof the criminal procedure)',
                                           lower_text)))
    else:
        CrPC_number.append({})
    if 'penal code' in lower_text or 'ipc' in lower_text :
        IPC_number.append(set(re.findall(r'(?<=ipc\s)[0-9]+|[0-9]+(?=\sipc) | '
                                          r'(?<=of the penal\s)[0-9]+|[0-9]+(?=\sof the penal) | '
                                          r'(?<=of the indian penal\s)[0-9]+|[0-9]+(?=\sof the indian penal)',
                                          lower_text)))
    else:
        IPC_number.append({})
    if  'constitution' in lower_text:
        Constitution_number.append(set(re.findall(r'(?<=of the constitution\s)[0-9]+|[0-9]+(?=\sof the constitution) | '
                                                   r'(?<=of the indian constitution\s)[0-9]+|[0-9]+(?=\sof the indian constitution)',
                                                   lower_text)))
    else:
        Constitution_number.append({})

In [9]:
data['Constitution_number'] = Constitution_number
data['IPC_number'] = IPC_number
data['CrPC_number'] = CrPC_number
data

,article_number,text,label,split,Constitution_number,IPC_number,CrPC_number
0,0,"F. NARIMAN, J. Leave granted. In 2008, the Pu...",1,train,"{16 , 14 }",{},{}
1,1,"S. THAKUR, J. Leave granted. These appeals ar...",0,train,"{309 , 142 , 368 , 226 , 141 }",{},{}
2,2,"Markandey Katju, J. Leave granted. Heard lear...",1,train,{},{},{125 }
3,3,"ALTAMAS KABIR,J. Leave granted. The question ...",1,train,{226 },"{406 , 302 , 420 , 326 }","{320 , 482 }"
4,4,"CIVIL APPEAL NO. 598 OF 2007 K. MATHUR, J. Th...",1,train,{225 },{},{}
...,...,...,...,...,...,...,...
6071,6071,civil appellate jurisdiction civil appeal numb...,1,dev,{},{},{}
6072,6072,criminal appellate jurisdiction special leave\...,0,dev,{},{},{}
6073,6073,civil appellate jurisdiction civil appeal numb...,0,dev,{},{},{}
6074,6074,civil appellate jurisdiction civil appeal numb...,1,dev,{},{},{}


In [14]:
include=0
for const, ipc, cr in zip(data['Constitution_number'], data['IPC_number'], data['CrPC_number']):
    if not (len(const)!=0 or len(ipc)!=0 or len(cr)!=0):
        include+=1

print("Has at least one reference to either Constitution, either IPC, or CrPC: ", include, "(",100*include/len(data['text']),"%)")

Has at least one reference to either Constitution, either IPC, or CrPC:  3556 ( 58.525345622119815 %)


# Constitution

In [15]:
data

,article_number,text,label,split,Constitution_number,IPC_number,CrPC_number
0,0,"F. NARIMAN, J. Leave granted. In 2008, the Pu...",1,train,"{16 , 14 }",{},{}
1,1,"S. THAKUR, J. Leave granted. These appeals ar...",0,train,"{309 , 142 , 368 , 226 , 141 }",{},{}
2,2,"Markandey Katju, J. Leave granted. Heard lear...",1,train,{},{},{125 }
3,3,"ALTAMAS KABIR,J. Leave granted. The question ...",1,train,{226 },"{406 , 302 , 420 , 326 }","{320 , 482 }"
4,4,"CIVIL APPEAL NO. 598 OF 2007 K. MATHUR, J. Th...",1,train,{225 },{},{}
...,...,...,...,...,...,...,...
6071,6071,civil appellate jurisdiction civil appeal numb...,1,dev,{},{},{}
6072,6072,criminal appellate jurisdiction special leave\...,0,dev,{},{},{}
6073,6073,civil appellate jurisdiction civil appeal numb...,0,dev,{},{},{}
6074,6074,civil appellate jurisdiction civil appeal numb...,1,dev,{},{},{}


In [16]:
constitution

,Articles,number,final_Articles
number,,,
1,"1. Name and territory of the Union\n(1) India,...",1,"Name and territory of the Union. India, that i..."
10,10. Continuance of the rights of citizenship E...,10,Continuance of the rights of citizenship Every...
100,"100. Voting in Houses, power of Houses to act ...",100,"Voting in Houses, power of Houses to act notwi..."
101,101. Vacation of seats\n(1) No person shall be...,101,Vacation of seats. No person shall be a member...
102,102. Disqualification s for membership\n(1) A ...,102,Disqualification s for membership. A person sh...
...,...,...,...
95,95. Power of the Deputy Speaker or other perso...,95,Power of the Deputy Speaker or other person to...
96,96. The Speaker or the Deputy Speaker not to p...,96,The Speaker or the Deputy Speaker not to presi...
97,97. Salaries and allowances of the Chairman an...,97,Salaries and allowances of the Chairman and De...


In [57]:
# constitution['clear_articles'] = constitution['Articles'].apply(
#     lambda x: ''.join(x.split('. ')[1:]).replace('\n', '')
# )
# constitution

,Articles,number,final_Articles,clear_articles
number,,,,
1,"1. Name and territory of the Union\n(1) India,...",1,"Name and territory of the Union. India, that i...","Name and territory of the Union(1) India, that..."
10,10. Continuance of the rights of citizenship E...,10,Continuance of the rights of citizenship Every...,Continuance of the rights of citizenship Every...
100,"100. Voting in Houses, power of Houses to act ...",100,"Voting in Houses, power of Houses to act notwi...","Voting in Houses, power of Houses to act notwi..."
101,101. Vacation of seats\n(1) No person shall be...,101,Vacation of seats. No person shall be a member...,Vacation of seats(1) No person shall be a memb...
102,102. Disqualification s for membership\n(1) A ...,102,Disqualification s for membership. A person sh...,Disqualification s for membership(1) A person ...
...,...,...,...,...
95,95. Power of the Deputy Speaker or other perso...,95,Power of the Deputy Speaker or other person to...,Power of the Deputy Speaker or other person to...
96,96. The Speaker or the Deputy Speaker not to p...,96,The Speaker or the Deputy Speaker not to presi...,The Speaker or the Deputy Speaker not to presi...
97,97. Salaries and allowances of the Chairman an...,97,Salaries and allowances of the Chairman and De...,Salaries and allowances of the Chairman and De...


In [17]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

import string
def process_sentence(row):
    sent = row['Articles'].translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(sent.lower())
    row['article_number'] = tokens[0]
    row['article_tokenized'] = tokens[1:]
    row['tokens_length'] = len(tokens[1:])
    return row

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/amirdjanibekov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
def process_text(row):
    initial_text = row['text'].translate(str.maketrans('', '', string.punctuation))
    row['processed_text'] = initial_text
    return row

data = data.apply(process_text, axis=1)

In [19]:
data

,article_number,text,label,split,Constitution_number,IPC_number,CrPC_number,processed_text
0,0,"F. NARIMAN, J. Leave granted. In 2008, the Pu...",1,train,"{16 , 14 }",{},{},F NARIMAN J Leave granted In 2008 the Punjab ...
1,1,"S. THAKUR, J. Leave granted. These appeals ar...",0,train,"{309 , 142 , 368 , 226 , 141 }",{},{},S THAKUR J Leave granted These appeals are di...
2,2,"Markandey Katju, J. Leave granted. Heard lear...",1,train,{},{},{125 },Markandey Katju J Leave granted Heard learned...
3,3,"ALTAMAS KABIR,J. Leave granted. The question ...",1,train,{226 },"{406 , 302 , 420 , 326 }","{320 , 482 }",ALTAMAS KABIRJ Leave granted The question whe...
4,4,"CIVIL APPEAL NO. 598 OF 2007 K. MATHUR, J. Th...",1,train,{225 },{},{},CIVIL APPEAL NO 598 OF 2007 K MATHUR J This a...
...,...,...,...,...,...,...,...,...
6071,6071,civil appellate jurisdiction civil appeal numb...,1,dev,{},{},{},civil appellate jurisdiction civil appeal numb...
6072,6072,criminal appellate jurisdiction special leave\...,0,dev,{},{},{},criminal appellate jurisdiction special leave\...
6073,6073,civil appellate jurisdiction civil appeal numb...,0,dev,{},{},{},civil appellate jurisdiction civil appeal numb...
6074,6074,civil appellate jurisdiction civil appeal numb...,1,dev,{},{},{},civil appellate jurisdiction civil appeal numb...


In [20]:
def add_constitution(row):
    row['constitution_article'] = ''

    for ent in row['Constitution_number']:
        normal_ent = ent.replace(' ', '')  # we replace an empty space with zero space since in the previous step where article number is parsed the int type conversion was not applied. this means that all numbers are in str format, and we continue work with this format
        text_constitution = constitution[constitution['number'] == normal_ent]['final_Articles'].values
        row['constitution_article'] = row['constitution_article'] + '~~' + ''.join(text_constitution).translate(str.maketrans('', '', string.punctuation))
    return row

data = data.apply(add_constitution, axis=1)

In [21]:
data[['processed_text', 'constitution_article', 'label', 'split']]

,processed_text,constitution_article,label,split
0,F NARIMAN J Leave granted In 2008 the Punjab ...,~~Equality of opportunity in matters of public...,1,train
1,S THAKUR J Leave granted These appeals are di...,~~Recruitment and conditions of service of per...,0,train
2,Markandey Katju J Leave granted Heard learned...,,1,train
3,ALTAMAS KABIRJ Leave granted The question whe...,~~Power of High Courts to issue certain writs ...,1,train
4,CIVIL APPEAL NO 598 OF 2007 K MATHUR J This a...,~~Jurisdiction of existing High Courts Subject...,1,train
...,...,...,...,...
6071,civil appellate jurisdiction civil appeal numb...,,1,dev
6072,criminal appellate jurisdiction special leave\...,,0,dev
6073,civil appellate jurisdiction civil appeal numb...,,0,dev
6074,civil appellate jurisdiction civil appeal numb...,,1,dev


# Code of Criminal

In [22]:
data['CrPC_number']

0                 {}
1                 {}
2             {125 }
3       {320 , 482 }
4                 {}
            ...     
6071              {}
6072              {}
6073              {}
6074              {}
6075              {}
Name: CrPC_number, Length: 6076, dtype: object

In [25]:
crpc

,article,full_text
0,1,Short title extent and commencement. This Act ...
1,2,"Definitions. In this Code, unless the context ..."
2,3,"Construction of references. In this Code,- In ..."
3,4,Trial of offences under the Indian Penal Code ...
4,5,"Saving. Nothing contained in this Code shall, ..."
...,...,...
497,480,Practising pleader not to sit as Magistrate in...
498,481,Public servant concerned in sale not to purcha...
499,482,Saving of inherent powers of High Court. Nothi...
500,483,Duty of High Court to exercise continuous supe...


In [26]:
def add_crpc(row):
    row['crpc_article'] = ''
    for ent in row['CrPC_number']:
        normal_ent = ent.replace(' ', '')
        text_constitution = crpc[crpc['article'] == normal_ent]['full_text'].values
        row['crpc_article'] = row['crpc_article'] + '~~' + ' '.join(text_constitution).translate(str.maketrans('', '', string.punctuation)).replace('\t', '')
    return row

data = data.apply(add_crpc, axis=1)

In [27]:
data[['processed_text', 'constitution_article', 'crpc_article', 'label', 'split']]

,processed_text,constitution_article,crpc_article,label,split
0,F NARIMAN J Leave granted In 2008 the Punjab ...,~~Equality of opportunity in matters of public...,,1,train
1,S THAKUR J Leave granted These appeals are di...,~~Recruitment and conditions of service of per...,,0,train
2,Markandey Katju J Leave granted Heard learned...,,~~Order for maintenance of wives children and ...,1,train
3,ALTAMAS KABIRJ Leave granted The question whe...,~~Power of High Courts to issue certain writs ...,~~Compounding of offences The offences punisha...,1,train
4,CIVIL APPEAL NO 598 OF 2007 K MATHUR J This a...,~~Jurisdiction of existing High Courts Subject...,,1,train
...,...,...,...,...,...
6071,civil appellate jurisdiction civil appeal numb...,,,1,dev
6072,criminal appellate jurisdiction special leave\...,,,0,dev
6073,civil appellate jurisdiction civil appeal numb...,,,0,dev
6074,civil appellate jurisdiction civil appeal numb...,,,1,dev


# IPC

In [28]:
IPC

,section_number,section_text
0,1,This Act shall be called the Indian Penal Code...
1,2,Every person shall be liable to punishment und...
2,3,"Any person liable, by any Indian law, to be tr..."
3,4,The provisions of this Code apply also to any ...
4,5,Nothing in this Act shall affect the provision...
...,...,...
554,507,Whoever commits the offence of criminal intimi...
555,508,Whoever voluntarily causes or attempts to caus...
556,509,"Whoever, intending to insult the modesty of an..."
557,510,"Whoever, in a state of intoxication, appears i..."


In [30]:
data

,article_number,text,label,split,Constitution_number,IPC_number,CrPC_number,processed_text,constitution_article,crpc_article
0,0,"F. NARIMAN, J. Leave granted. In 2008, the Pu...",1,train,"{16 , 14 }",{},{},F NARIMAN J Leave granted In 2008 the Punjab ...,~~Equality of opportunity in matters of public...,
1,1,"S. THAKUR, J. Leave granted. These appeals ar...",0,train,"{309 , 142 , 368 , 226 , 141 }",{},{},S THAKUR J Leave granted These appeals are di...,~~Recruitment and conditions of service of per...,
2,2,"Markandey Katju, J. Leave granted. Heard lear...",1,train,{},{},{125 },Markandey Katju J Leave granted Heard learned...,,~~Order for maintenance of wives children and ...
3,3,"ALTAMAS KABIR,J. Leave granted. The question ...",1,train,{226 },"{406 , 302 , 420 , 326 }","{320 , 482 }",ALTAMAS KABIRJ Leave granted The question whe...,~~Power of High Courts to issue certain writs ...,~~Compounding of offences The offences punisha...
4,4,"CIVIL APPEAL NO. 598 OF 2007 K. MATHUR, J. Th...",1,train,{225 },{},{},CIVIL APPEAL NO 598 OF 2007 K MATHUR J This a...,~~Jurisdiction of existing High Courts Subject...,
...,...,...,...,...,...,...,...,...,...,...
6071,6071,civil appellate jurisdiction civil appeal numb...,1,dev,{},{},{},civil appellate jurisdiction civil appeal numb...,,
6072,6072,criminal appellate jurisdiction special leave\...,0,dev,{},{},{},criminal appellate jurisdiction special leave\...,,
6073,6073,civil appellate jurisdiction civil appeal numb...,0,dev,{},{},{},civil appellate jurisdiction civil appeal numb...,,
6074,6074,civil appellate jurisdiction civil appeal numb...,1,dev,{},{},{},civil appellate jurisdiction civil appeal numb...,,


In [31]:
def add_crpc(row):
    row['ipc_article'] = ''
    for ent in row['IPC_number']:
        normal_ent = ent.replace(' ', '')
        text_constitution = IPC[IPC['section_number'] == normal_ent]['section_text'].values
        row['ipc_article'] = row['ipc_article'] + '~~' + ' '.join(text_constitution).translate(str.maketrans('', '', string.punctuation)).replace('\t', '')
    return row

data = data.apply(add_crpc, axis=1)

In [32]:
data[['processed_text', 'constitution_article', 'crpc_article', 'ipc_article', 'label', 'split']]

,processed_text,constitution_article,crpc_article,ipc_article,label,split
0,F NARIMAN J Leave granted In 2008 the Punjab ...,~~Equality of opportunity in matters of public...,,,1,train
1,S THAKUR J Leave granted These appeals are di...,~~Recruitment and conditions of service of per...,,,0,train
2,Markandey Katju J Leave granted Heard learned...,,~~Order for maintenance of wives children and ...,,1,train
3,ALTAMAS KABIRJ Leave granted The question whe...,~~Power of High Courts to issue certain writs ...,~~Compounding of offences The offences punisha...,~~Whoever commits criminal breach of trust sha...,1,train
4,CIVIL APPEAL NO 598 OF 2007 K MATHUR J This a...,~~Jurisdiction of existing High Courts Subject...,,,1,train
...,...,...,...,...,...,...
6071,civil appellate jurisdiction civil appeal numb...,,,,1,dev
6072,criminal appellate jurisdiction special leave\...,,,,0,dev
6073,civil appellate jurisdiction civil appeal numb...,,,,0,dev
6074,civil appellate jurisdiction civil appeal numb...,,,,1,dev


# Done preprocessing

In [33]:
processed_data = data[['article_number', 'processed_text', 'constitution_article', 'crpc_article', 'ipc_article', 'label', 'split']]
processed_data

,article_number,processed_text,constitution_article,crpc_article,ipc_article,label,split
0,0,F NARIMAN J Leave granted In 2008 the Punjab ...,~~Equality of opportunity in matters of public...,,,1,train
1,1,S THAKUR J Leave granted These appeals are di...,~~Recruitment and conditions of service of per...,,,0,train
2,2,Markandey Katju J Leave granted Heard learned...,,~~Order for maintenance of wives children and ...,,1,train
3,3,ALTAMAS KABIRJ Leave granted The question whe...,~~Power of High Courts to issue certain writs ...,~~Compounding of offences The offences punisha...,~~Whoever commits criminal breach of trust sha...,1,train
4,4,CIVIL APPEAL NO 598 OF 2007 K MATHUR J This a...,~~Jurisdiction of existing High Courts Subject...,,,1,train
...,...,...,...,...,...,...,...
6071,6071,civil appellate jurisdiction civil appeal numb...,,,,1,dev
6072,6072,criminal appellate jurisdiction special leave\...,,,,0,dev
6073,6073,civil appellate jurisdiction civil appeal numb...,,,,0,dev
6074,6074,civil appellate jurisdiction civil appeal numb...,,,,1,dev


In [34]:
processed_data['article_number'].value_counts()

0       1
4047    1
4056    1
4055    1
4054    1
       ..
2023    1
2022    1
2021    1
2020    1
6075    1
Name: article_number, Length: 6076, dtype: int64

In [35]:
processed_data.to_csv('processed_data_prime.csv', index=False)